In [1]:
import json
import numpy as np
from tqdm import tqdm
from PIL import Image as PIL_Image
import torch
from PIL import Image
import open_clip
from torch.nn.functional import cosine_similarity as cos_sim

In [2]:
DATA_PATH = '/local/home/stuff/visual-genome/'
objects = json.load(open(DATA_PATH + 'objects.json'))
image_ids = [o['image_id'] for o in objects]
def get_image(image_id):
    try:
        image = PIL_Image.open(DATA_PATH + 'VG_100K/' + str(image_id) + '.jpg')
    except:
        image = PIL_Image.open(DATA_PATH + 'VG_100K_2/' + str(image_id) + '.jpg')
    return image
shuffled_image_ids = np.random.permutation(image_ids)
del image_ids
image_id_corpus = shuffled_image_ids[:10000]

In [3]:
# objects[0]

In [4]:
region_descriptions = json.load(open(DATA_PATH + 'region_descriptions.json'))
print("loaded region descriptions")
attributes = json.load(open(DATA_PATH + 'attributes.json'))
print("loaded attributes")
relationships = json.load(open(DATA_PATH + 'relationships.json'))
print("loaded relationships")

loaded region descriptions
loaded attributes
loaded relationships


In [5]:
# apparently the format is different than state in the website description

# the attributes always contain the objects
# attributes[0]['attributes'][0]['attributes']

In [6]:
# attributes[0]

In [7]:
# attributes2 = {
#     image['image_id']: 
#             [attr for obj in image['attributes'] for attr in obj] for image in attributes}
# print(attributes2[2][0])

In [8]:
# count = 0
# for image in attributes:
#     count+=1
#     image_id = image['image_id']
#     for obj in image['attributes']:
#         try:
#             obj_attributes = obj['attributes']
#         except:
#             print(count)
#             print(obj)
#             raise Exception("no attributes")
# print(attributes[5]['attributes'][0]['attributes'])
# attribute_list = [[obj['attributes'] for obj in image['attributes']] for image in attributes]
# print(attribute_list[5])
# print(attributes[5]['attributes'][0]['names'])
# name_list = [name for image in attributes for obj in image['attributes'] for name in obj['names']]

In [9]:
region_descriptions2 = {r['id']: [region['phrase'] for region in  r['regions']] for r in region_descriptions}
# object_descriptions2 = {r['image_id']: [object['name'] for object in  r['objects']] for r in region_descriptions}
# attributes2 = {
#     image['image_id']: 
#         [attr + " " + name 
#             for attr_desc_list in image['attributes'] for name in attr_desc_list['names'] for attr in attr_desc_list['attributes'] ] for image in attributes}
# relationships2 = {r['image_id']: [relationship['subject']['name'] + " " + relationship['predicate'] + " " + relationship['object']['name'] for relationship in  r['relationships']] for r in relationships}


In [10]:
print(region_descriptions2[55])

['Flowers hanging from the pole', 'Trash can beside the street', 'Bikes parked beside the pole', 'White bag being carried', 'Bus on the street', 'Bike parked beside the building', 'Vehicle parked on the sidewalk', 'windows in the building', 'Line on the street', 'Person wearing a pink shirt', 'the bikes are parked', 'the bus is colorful', 'the buildings in the city', 'the flowers hanging on the pole', 'the street light is tall', 'the wheels under the bus', 'the woman and child holding balloons', 'a colorful public bus', 'a person in a white shirt', 'a person in a white shirt', 'a person in a black shirt', 'a child wearing white', 'a small outdoor lamp', 'a white road bicycle', 'a white and red bicycle', 'a public garbage can', 'a person in a pink shirt', 'square window on building', 'square window on building', 'square window on building', 'square window on building', 'square window on building', 'square window on building', 'square window on building', 'bicycle parked against tree', '

In [11]:
device = 3

In [12]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-bigG-14', pretrained='laion2b_s39b_b160k')
model.to(device)
tokenizer = open_clip.get_tokenizer('ViT-bigG-14')

In [13]:
def encode_images(image_ids):
    images = [get_image(image_id) for image_id in image_ids]
    preprocessed_images = [preprocess(image) for image in images]
    image_input = torch.tensor(np.stack(preprocessed_images)).to(device)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_featuress = model.encode_image(image_input) # .float()
        image_featuress = [image_features/image_features.norm(dim=-1, keepdim=True) for image_features in image_featuress]
    return image_featuress

def encode_text(text):
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(tokenizer(text).to(device)) #.float()
        text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features

In [14]:
print_feq = 1

def chunker(seq, size):
    return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
print("encoding images...")
encoded_image_corpus = []
for i,batch in enumerate(chunker(image_id_corpus, 100)):
    batch_encoded = encode_images(batch)
    encoded_image_corpus.extend(batch_encoded)
    if i % print_feq == 0:
        print(i,end='\r')
print("done encoding images")

ecoded_imgcorp_dict = {image_id: encoded_image.cpu() for image_id, encoded_image in zip(image_id_corpus, encoded_image_corpus)}
del encoded_image_corpus

encoding images...
done encoding images


In [15]:
from random import randint
# create dataset

# load images, get embeddings

# load descriptions

# image_id_corpus = ...

query_ids = np.random.choice(image_id_corpus, 10000)
print_feq = 1

def chunker(seq, size):
    return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
# print("encoding images...")
# encoded_image_corpus = []
# for i,batch in enumerate(chunker(image_id_corpus, 10)):
#     batch_encoded = encode_images(batch)
#     encoded_image_corpus.extend(batch_encoded)
#     if i % print_feq == 0:
#         print(i,end='\r')
# print("done encoding images")

# ecoded_imgcorp_dict = {image_id: encoded_image for image_id, encoded_image in zip(image_id_corpus, encoded_image_corpus)}
# del encoded_image_corpus


def get_descriptions(query_id):
    # objects = 
    # relations = ...
    # thirdone = ...
    # obj_descs = [...]
    # rel_descs = [...]
    # thirone_descs = [...]
    descs = region_descriptions2[query_id] # obj_descs + rel_descs + thirone_descs
    return descs

print("getting all descriptions...")
all_descs = []
for query_id in tqdm(image_id_corpus):
    descs = get_descriptions(query_id)
    all_descs.extend(descs)
all_descs = list(set(all_descs))
print("done getting all descriptions")

print("getting query descriptions...")
query_descs_true = []
query_descs_false = []
for query_id in tqdm(query_ids):
    descs = get_descriptions(query_id)
    true_idx = randint(0, len(descs)-1)
    description = descs[true_idx]
    query_descs_true.append(description)
    false_idx = randint(0, len(all_descs)-1)
    query_descs_false.append(all_descs[false_idx])
print("done getting query descriptions")

print("encoding descriptions...")
encoded_descs_true = []
encoded_descs_false = []
for descriptions in chunker(query_descs_true, 100):
    batch_encoded = encode_text(descriptions)
    encoded_descs_true.extend(batch_encoded)
for descriptions in chunker(query_descs_false, 100):
    batch_encoded = encode_text(descriptions)
    encoded_descs_false.extend(batch_encoded)
print("done encoding descriptions")

# filtering out images that have the object is pointless at this stage, because the descriptions are not exactly the same for similar objects
image_ids = np.random.choice(image_id_corpus, 10000*50).reshape(10000, 50)
encoded_descs_true = [desc.cpu() for desc in encoded_descs_true]
encoded_descs_false = [desc.cpu() for desc in encoded_descs_false]

getting all descriptions...


100%|██████████| 10000/10000 [00:00<00:00, 186008.43it/s]


done getting all descriptions
getting query descriptions...


100%|██████████| 10000/10000 [00:00<00:00, 346949.24it/s]

done getting query descriptions
encoding descriptions...


done encoding descriptions


In [16]:
dataset_x = []
dataset_y = []
for query_id,desc_true,desc_false,comp_ids  in tqdm(zip(query_ids, encoded_descs_true, encoded_descs_false, image_ids)):
    query_enc = ecoded_imgcorp_dict[query_id]
    comp_encs = [ecoded_imgcorp_dict[comp_id] for comp_id in comp_ids]
    query_similarity_true = cos_sim(query_enc, desc_true, dim=-1)
    query_similarity_false = cos_sim(query_enc, desc_false, dim=-1)
    comp_similarities_true = [cos_sim(comp_enc, desc_true, dim=-1) for comp_enc in comp_encs]
    comp_similarities_false = [cos_sim(comp_enc, desc_false, dim=-1) for comp_enc in comp_encs]
    dataset_x.append( (comp_similarities_true,query_similarity_true) )
    dataset_y.append(1)
    dataset_x.append( (comp_similarities_false,query_similarity_false) )
    dataset_y.append(0)

10000it [00:22, 439.06it/s]


In [17]:
def get_percentile(comp_similarities, query_similarity):
    n_higher = sum([similarity >= query_similarity for similarity in comp_similarities])
    return n_higher / len(image_ids)

def get_std_distance(comp_similarities, query_similarity):
    mean = np.mean(comp_similarities)
    std = np.std(comp_similarities)
    return (query_similarity - mean) / std

X_percentiles = [get_percentile(comp_similarities, query_similarity) for comp_similarities, query_similarity in dataset_x]
X_std_distances = [get_std_distance(comp_similarities, query_similarity) for comp_similarities, query_similarity in dataset_x]
Y = dataset_y

In [18]:
from sklearn.svm import LinearSVC
import numpy as np
toy_data_x = np.array([0.0,1.0,2.0,3.0]).reshape(-1,1)
toy_data_y = [0,0,1,1]
svm = LinearSVC(loss = 'hinge', C = 10000)
svm.fit(toy_data_x, toy_data_y)
svm.predict(np.array([0.5,1.49,1.51, 2.5]).reshape(-1,1))


array([0, 0, 1, 1])

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(torch.stack(X_percentiles).reshape(-1,1), Y, test_size=0.2, random_state=42)
svm_percentile = LinearSVC(loss = 'hinge', C = 10000, max_iter=1000000)
svm_percentile.fit(X_train, Y_train)
accuracy_percentile = svm_percentile.score(X_test, Y_test)
print(accuracy_percentile)

X_train, X_test, Y_train, Y_test = train_test_split(torch.stack(X_std_distances).reshape(-1,1), Y, test_size=0.2, random_state=42)
svm_std_distance = LinearSVC(loss = 'hinge', C = 10, max_iter=1000000)
svm_std_distance.fit(X_train, Y_train)
accuracy_std_distance = svm_std_distance.score(X_test, Y_test)
print(accuracy_std_distance)

0.81375
0.83025


In [20]:
print(svm_percentile.intercept_)
print(svm_percentile.coef_)

print(svm_std_distance.intercept_)
print(svm_std_distance.coef_)


[1.09524066]
[[-952.40658825]]
[-1.12951556]
[[0.92780488]]
